In [1]:
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [2]:
# Import data.
X_test = []
for p in range(11):
    tmp = pd.read_csv("data\\clean\\clean_data"+str(p)+".csv")
    X_test.append(tmp['text'])
X_test

[0          break kamala harris give update russia ukraine...
 1          ukraine anti war protesters fill streets mosco...
 2          indiana house unanimously support sanction rus...
 3          corrupt billionaires russia also sanction acco...
 4          flash russia ministry defence confirm water fl...
                                  ...                        
 1446025    predsjednik usambmne stop russian aggression u...
 1446026    mustafasentop rterdogan emineerdogan stop russ...
 1446027    unacceptable russia european countries host nu...
 1446028    point convince fhking idea wtf happen ukrainei...
 1446029    ready impose sanction russia invade ukraine ni...
 Name: text, Length: 1446030, dtype: object,
 0          istandwithputin ukrainerussiawar muslimslifema...
 1          new weapons germany promise anti tank stingers...
 2          break thursday london russianukrainianwar russ...
 3          energodar people take street stop tank enter c...
 4          irpen russian

In [3]:
# Functions to clean training data.
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt   

# Functions to clean training data.
def reprocessing(df):
    df['tidy_Tweet'] = np.vectorize(remove_pattern)(df['Tweet'], "@[\w]*") 
    df['tidy_Tweet'] = df['tidy_Tweet'].str.replace("[^a-zA-Z#]", " ")
    df['tidy_Tweet'] = df['tidy_Tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
    df['tidy_Tweet'] = df['tidy_Tweet'].apply(lambda x: ' '.join([word.lower() for word in x.split() if word not in stop_words]))
    lmtzr = WordNetLemmatizer()
    df['tidy_Tweet'] = df['tidy_Tweet'].apply(lambda x: ' '.join([lmtzr.lemmatize(w,'v') for w in x.split()]))
    return df['tidy_Tweet']

In [4]:
# Import training data.
df_train = pd.read_csv('data/training/2018-E-c-En-train.txt', sep="\t")
df_dev = pd.read_csv('data/training/2018-E-c-En-dev.txt', sep="\t")
X_train = reprocessing(df_train)
X_dev = reprocessing(df_dev)
#X_test1 = reprocessing(df_test1)
#X_test2 = reprocessing(df_test2)
#X_test3 = reprocessing(df_test3)

<ipython-input-3-9aa55a21631d>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tidy_Tweet'] = df['tidy_Tweet'].str.replace("[^a-zA-Z#]", " ")


In [5]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model, load_model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

In [6]:
# Get tokenizer.
X_train = df_train.tidy_Tweet
y_train = df_train.drop(labels = ['ID','Tweet','tidy_Tweet'], axis=1)
X_dev = df_dev.tidy_Tweet
y_dev = df_dev.drop(labels = ['ID','Tweet','tidy_Tweet'], axis=1)
                    

print(X_train.shape, X_dev.shape)
X_train_dev = pd.concat([X_train, X_dev])
#print(type(X_train), type(X_dev))
print(X_train_dev.shape)
#print(newX[:10])
tokenizer = Tokenizer(num_words=10000)
#fix
#tokenizer.fit_on_texts(X_train)
#tokenizer.fit_on_texts(X_test)
 
tokenizer.fit_on_texts(X_train_dev)

maxlen = 150

(6838,) (886,)
(7724,)


In [7]:
# Tokenize test data.
def format(value):
    value = str(value)
    return value

for p in range(11):
    X_test[p] = X_test[p].apply(format)
    X_test[p] = tokenizer.texts_to_sequences(X_test[p])
    X_test[p] = pad_sequences(X_test[p], padding='post', maxlen=maxlen)
X_test

[array([[ 176, 4037,   37, ...,    0,    0,    0],
        [1545,  327, 2832, ...,    0,    0,    0],
        [ 216,  244, 1103, ...,    0,    0,    0],
        ...,
        [2574, 1103, 5805, ...,    0,    0,    0],
        [ 227, 1187,  550, ...,    0,    0,    0],
        [ 492, 1103, 6291, ...,    0,    0,    0]]),
 array([[ 560,  630, 1100, ...,    0,    0,    0],
        [  43, 4701, 3090, ...,    0,    0,    0],
        [ 176, 1807, 1629, ...,    0,    0,    0],
        ...,
        [ 882,  214,   37, ...,    0,    0,    0],
        [1438, 1316, 2887, ...,    0,    0,    0],
        [6305, 1438, 2587, ...,    0,    0,    0]]),
 array([[  19,  161,  165, ...,    0,    0,    0],
        [  20,   44,    5, ...,    0,    0,    0],
        [1037,  238,   55, ...,    0,    0,    0],
        ...,
        [1103,   77, 1438, ...,    0,    0,    0],
        [ 959, 1522,  218, ...,    0,    0,    0],
        [ 169,  572, 7980, ...,    0,    0,    0]]),
 array([[  11,   61, 1192, ...,    0,

In [8]:
# Load the model.
model = load_model('LSTM_model_ver1.h5')

In [10]:
# Use the model to predict emotions with test data.
# It takes a whole night to run.
predictions = []
for p in range(11):
    predictions.append(model.predict(X_test[p]))
    print(predictions[p].shape)
    print(type(predictions[p]))
    print(predictions[p])

(1446030, 11)
<class 'numpy.ndarray'>
[[0.34633648 0.28030637 0.42828614 ... 0.10801846 0.01176032 0.09215313]
 [0.96807754 0.03220016 0.9403208  ... 0.24193561 0.01701295 0.00244719]
 [0.44389555 0.25748459 0.2778663  ... 0.03980502 0.01690078 0.02290079]
 ...
 [0.8327918  0.12476858 0.5922745  ... 0.06051838 0.03591624 0.02045757]
 [0.2388528  0.2070958  0.2691454  ... 0.15090722 0.01000309 0.04911366]
 [0.5681063  0.17432553 0.6021971  ... 0.06210554 0.04669461 0.01554781]]
(2037917, 11)
<class 'numpy.ndarray'>
[[0.88527066 0.04289764 0.8830322  ... 0.2859093  0.0055612  0.00420982]
 [0.02962014 0.37367377 0.05599329 ... 0.08282399 0.03830448 0.25318122]
 [0.700282   0.09108692 0.57992125 ... 0.11604321 0.03283435 0.00682092]
 ...
 [0.9180803  0.0146217  0.920758   ... 0.30404523 0.00439483 0.00381306]
 [0.95501363 0.02283946 0.92116594 ... 0.23432896 0.00882611 0.00922626]
 [0.0329257  0.05575371 0.10330877 ... 0.11764643 0.05700156 0.02237356]]
(1832609, 11)
<class 'numpy.ndarray'

ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[150,32,200] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node ReverseV2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[functional_1/bidirectional/backward_lstm/PartitionedCall]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_predict_function_3996]

In [9]:
predictions = []
for p in range(2):
    predictions.append(model.predict(X_test[p+9]))
    print(predictions[p+9].shape)
    print(type(predictions[p+9]))
    print(predictions[p+9])

IndexError: list index out of range

In [18]:
X_test[10]

array([[8248, 3136,  586, ...,    0,    0,    0],
       [1103, 1394, 3837, ...,    0,    0,    0],
       [2909, 1816,  153, ...,    0,    0,    0],
       ...,
       [1497, 1103,  959, ...,    0,    0,    0],
       [5216, 1103, 2453, ...,    0,    0,    0],
       [7955, 1105, 2498, ...,    0,    0,    0]])

In [19]:
predictions.append(model.predict(X_test[10]))

In [20]:
np.savetxt("data\\Predictions_prob\\Predictions_prob10.csv",predictions[1], delimiter = ',')

In [13]:
# Save the prediction results in probablity form.
for p in range(9):
    print(predictions[p])
    np.savetxt("data\\Predictions_prob\\Predictions_prob"+str(p)+".csv",predictions[p], delimiter = ',') 

[[0.34633648 0.28030637 0.42828614 ... 0.10801846 0.01176032 0.09215313]
 [0.96807754 0.03220016 0.9403208  ... 0.24193561 0.01701295 0.00244719]
 [0.44389555 0.25748459 0.2778663  ... 0.03980502 0.01690078 0.02290079]
 ...
 [0.8327918  0.12476858 0.5922745  ... 0.06051838 0.03591624 0.02045757]
 [0.2388528  0.2070958  0.2691454  ... 0.15090722 0.01000309 0.04911366]
 [0.5681063  0.17432553 0.6021971  ... 0.06210554 0.04669461 0.01554781]]
[[0.88527066 0.04289764 0.8830322  ... 0.2859093  0.0055612  0.00420982]
 [0.02962014 0.37367377 0.05599329 ... 0.08282399 0.03830448 0.25318122]
 [0.700282   0.09108692 0.57992125 ... 0.11604321 0.03283435 0.00682092]
 ...
 [0.9180803  0.0146217  0.920758   ... 0.30404523 0.00439483 0.00381306]
 [0.95501363 0.02283946 0.92116594 ... 0.23432896 0.00882611 0.00922626]
 [0.0329257  0.05575371 0.10330877 ... 0.11764643 0.05700156 0.02237356]]
[[0.3373439  0.03122956 0.42114237 ... 0.94512194 0.00336546 0.0211693 ]
 [0.05181077 0.4174116  0.11466271 ... 

In [10]:
# Import prob data.
probs = []
for p in range(11):
    tmp = pd.read_csv("data\\Predictions_prob\\Predictions_prob"+str(p)+".csv").to_numpy()
    probs.append(tmp)
probs

[array([[0.96807754, 0.03220016, 0.94032079, ..., 0.24193561, 0.01701295,
         0.00244719],
        [0.44389555, 0.25748459, 0.2778663 , ..., 0.03980502, 0.01690078,
         0.02290079],
        [0.28190732, 0.02880788, 0.49999532, ..., 0.78023934, 0.02273583,
         0.00368017],
        ...,
        [0.83279181, 0.12476858, 0.59227449, ..., 0.06051838, 0.03591624,
         0.02045757],
        [0.2388528 , 0.2070958 , 0.2691454 , ..., 0.15090722, 0.01000309,
         0.04911366],
        [0.56810629, 0.17432553, 0.60219711, ..., 0.06210554, 0.04669461,
         0.01554781]]),
 array([[0.02962014, 0.37367377, 0.05599329, ..., 0.08282399, 0.03830448,
         0.25318122],
        [0.70028198, 0.09108692, 0.57992125, ..., 0.11604321, 0.03283435,
         0.00682092],
        [0.05319774, 0.04661748, 0.1889489 , ..., 0.8102861 , 0.02710831,
         0.01591599],
        ...,
        [0.91808027, 0.0146217 , 0.92075801, ..., 0.30404523, 0.00439483,
         0.00381306],
        [0.9

In [12]:
# Use 0.37 as the threshold to make predictions binary.
def threshold(value):
    if float(value)>=0.37:
        return int(1)
    else:
        return int(0)
    

for p in range(11):
    for pred in probs[p]:
        for emotion in range(len(pred)):
            pred[emotion] = int(threshold(pred[emotion]))
probs

[array([[1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.]]),
 array([[0., 1., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 1., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 1., 0., 0.]]),
 arr

In [14]:
for p in range(11):
    np.savetxt("data\\Predictions\\Predictions"+str(p)+".csv",probs[p], delimiter = ',') 